# PennyLane internals: Quantum Tapes
## Christina Lee

What is a tape?

<img src="./pictures/tape_def2.jpeg" width="700" style="margin-left: auto; margin-right: auto"/>

Think this type of tape:

<img src="./pictures/casette_tape.jpg" width="300" style="margin-left: auto; margin-right: auto"/>

Inspired by tensorflow tapes:

In [2]:
import tensorflow as tf

x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x**2

dy_dx = tape.gradient(y, x)
dy_dx.numpy()

6.0

So let's see them!

In [3]:
import pennylane as qml
from pennylane import numpy as np

In [37]:
with qml.tape.QuantumTape() as tape:
    qml.PauliX(wires=0)
    qml.RY(0.1, wires=1)
    qml.CRZ(0.2, wires=(0, 1))
    qml.expval(qml.PauliZ(1)) 

In [5]:
tape.operations

[PauliX(wires=[0]), RY(0.1, wires=[1]), CRZ(0.2, wires=[0, 1])]

In [6]:
tape.measurements

[expval(PauliZ(wires=[1]))]

In [7]:
tape.get_parameters()

[0.1, 0.2]

In [8]:
tape.trainable_params

{0, 1}

In [9]:
tape.wires

<Wires = [0, 1]>

But I've never seen one before?

In [10]:
dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)
def circuit(x):
    qml.PauliX(wires=0)
    qml.RY(x[0], wires=1)
    qml.CRZ(x[1], wires=(0, 1))
    return qml.expval(qml.PauliZ(1))

QNode's create them internally, and we can access them via the `qtape` attribute.

But where is the `qtape` now?

In [11]:
print(circuit.qtape)

None


The qtape is created during execution!

In [12]:
x = np.array([0.1, 0.2])

circuit(x)

print(circuit.qtape)

<JacobianTape: wires=[0, 1], params=2>


In [13]:
circuit.qtape.operations

[PauliX(wires=[0]),
 RY(tensor(0.1, requires_grad=True), wires=[1]),
 CRZ(tensor(0.2, requires_grad=True), wires=[0, 1])]

But I just want the tape.  I don't want to execute it.

You can just call `construct`:

In [32]:
circuit.construct(([0.6, 0.7], ) , {} )

circuit.qtape.operations

[PauliX(wires=[0]),
 RY(tensor(0.6, requires_grad=True), wires=[1]),
 CRZ(tensor(0.7, requires_grad=True), wires=[0, 1])]

By default, tapes are constructed on every execution, in case of classical logic:

In [14]:
%timeit circuit(x)

1.45 ms ± 486 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


If you know the circuit isn't changing structure, you can make the qnode **immutable** for a slight performance boost:

In [15]:
circuit.mutable = False
%timeit circuit(x)
circuit.mutable = True

836 µs ± 27.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


What about templates?

In [20]:
rng = np.random.default_rng(seed=42)
params = rng.random(qml.templates.BasicEntanglerLayers.shape(n_wires=2, n_layers=2))

with qml.tape.QuantumTape() as tape:
    qml.templates.BasicEntanglerLayers(params, wires=(0,1))

In [21]:
tape.operations

[BasicEntanglerLayers(tensor([[0.77395605, 0.43887844],
         [0.85859792, 0.69736803]], requires_grad=True), wires=[0, 1])]

QNode's perform an expansion to tapes supported on devices:

In [23]:
new_tape = tape.expand()

new_tape.operations

[RX(tensor(0.77395605, requires_grad=True), wires=[0]),
 RX(tensor(0.43887844, requires_grad=True), wires=[1]),
 CNOT(wires=[0, 1]),
 RX(tensor(0.85859792, requires_grad=True), wires=[0]),
 RX(tensor(0.69736803, requires_grad=True), wires=[1]),
 CNOT(wires=[0, 1])]

Nice for seeing what the circuit is, but what is it good for?

How about custom tape transforms?

In [28]:
@qml.single_tape_transform
def convert_cnots(tape):
    # Loop through all items in the original tape
    for op in tape.operations + tape.measurements:

        # If it's a CNOT, replace it using the circuit identity
        if op.name == "CNOT":
            wires = op.wires
            qml.Hadamard(wires=wires[1])
            qml.CZ(wires=[wires[1], wires[0]])
            qml.Hadamard(wires=wires[1])

        # If it's not a CNOT, keep the operation as-is and apply it
        else:
            qml.apply(op)

In [30]:
converted_tape = convert_cnots(new_tape)

converted_tape.operations

[RX(tensor(0.77395605, requires_grad=True), wires=[0]),
 RX(tensor(0.43887844, requires_grad=True), wires=[1]),
 Hadamard(wires=[1]),
 CZ(wires=[1, 0]),
 Hadamard(wires=[1]),
 RX(tensor(0.85859792, requires_grad=True), wires=[0]),
 RX(tensor(0.69736803, requires_grad=True), wires=[1]),
 Hadamard(wires=[1]),
 CZ(wires=[1, 0]),
 Hadamard(wires=[1])]

Or writing your own device!

In [35]:
dev = qml.device('default.qubit', wires=2)

In [36]:
dev.execute(tape)

array([0.99500417])